<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%9D%D0%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Нейронные сети
####Вариант 5:
Просчитать одну итерацию цикла обучения по Δ-правилу однослойной бинарной
неоднородной нейронной сети, состоящей из 2 нейронов и имеющей функции активации: гиперболический тангенс ($k=2$) и пороговую функцию ($Т=0.5$). В качестве обучающей выборки использовать таблицу истинности для операций эквивалентности и конъюнкции (не использовать первую строчку таблицы). Синаптические веса задать случайным образом.

In [8]:
import numpy as np
import pandas as pd
import random
import copy
import math
import itertools as it

In [13]:
#Коэффициент обучения (точность) [0;1]
learningRate = 0.6
#Количество слоёв
layer = 1
#количество нейронов на каждом слое
neurons = 2

#--------------------------------------------------------------------

#функции активации
#гиперболический тангенс:
def hyperbolicTangentFunction(S, k = 2):
  return math.tanh(S / k)
#сигмоидная функция:
def sigmoidFunction(S, k = 2):
  return 1.0 / (1 + math.exp(- S * k))
#Пороговая функция:
def thresholdFunction(S, T = 0.5):
  return 1 if S >= T else 0

#--------------------------------------------------------------------

def randomVector(size = 2):
  return [random.uniform(0, 1) for i in range(size)]

def randomNumber():
  return random.uniform(0, 1)

def equiv(a,b): # эквивалентность ~
  rez = 0
  if((a == 1 and b == 1) or (a == 0 and b == 0)): rez = 1
  return rez

In [56]:
def deltaRule(data, level = 1, N = 2, nabla = 0.1):
  wi = [[randomNumber() for i in range(N)] for j in range(N)]         #синаптические веса
  si = [0] * N
  yi = [0] * N
  epsi = [0] * N

  level_i = 0
  k = 0
  while(level_i < level): #цикл по нейронным слоям
    while(k < len(data)): #цикл по строкам таблицы с данными
      eps = 0
      print("Матрица весов:")
      print(pd.DataFrame(wi))
      print("\nДля x = ( " + str(data[data.columns[0]][k]) + ", " + str(data[data.columns[1]][k]) + ' )')

      for i in range(N):
        for j in range(N):
          si[i] += data[data.columns[j]][k] * wi[j][i]

        yi[i] = hyperbolicTangentFunction(si[i])
        epsi[i] = data[data.columns[N + i]][k] - yi[i]

        print('S_' + str(i + 1) + ' = ' + str(si[i]))
        print('E_' + str(i + 1) + ' = ' +  str(yi[i]))
        print('EPS_' + str(i + 1) + ' = ' +  str(epsi[i]))

        eps += epsi[i]**2

      for i in range(N):
        for j in range(N):
          wi[j][i] = wi[j][i] - nabla * epsi[i] * data[data.columns[j]][k]

      print("\nСКО: " + str(eps) + "\n\n")

      k += 1
    level_i += 1


  #return 0

In [57]:
functions = [hyperbolicTangentFunction, thresholdFunction]

table = pd.DataFrame(it.product([0,1], repeat=2), columns=['X1', 'X2'])
table['Y1 = equivalence (~)'] = [equiv(table[table.columns[0]][i], table[table.columns[1]][i]) for i in range(2**neurons)]
table['Y2 = conjunction (^)'] = [(table[table.columns[0]][i] and table[table.columns[1]][i]) for i in range(2**neurons)]

deltaRule(table, layer, neurons, learningRate)

Матрица весов:
          0         1
0  0.653641  0.348315
1  0.142809  0.675874

Для x = ( 0, 0 )
S_1 = 0.0
E_1 = 0.0
EPS_1 = 1.0
S_2 = 0.0
E_2 = 0.0
EPS_2 = 0.0

СКО: 1.0


Матрица весов:
          0         1
0  0.653641  0.348315
1  0.142809  0.675874

Для x = ( 0, 1 )
S_1 = 0.14280911596700352
E_1 = 0.07128345028402011
EPS_1 = -0.07128345028402011
S_2 = 0.6758739505217749
E_2 = 0.32563436929961814
EPS_2 = -0.32563436929961814

СКО: 0.11111907275355445


Матрица весов:
          0         1
0  0.653641  0.348315
1  0.185579  0.871255

Для x = ( 1, 0 )
S_1 = 0.7964500082737953
E_1 = 0.37842918359505956
EPS_1 = -0.37842918359505956
S_2 = 1.024189383274424
E_2 = 0.4715756736769733
EPS_2 = -0.4715756736769733

СКО: 0.3655922630003145


Матрица весов:
          0         1
0  0.880698  0.631261
1  0.185579  0.871255

Для x = ( 1, 1 )
S_1 = 1.8627275968750385
E_1 = 0.7312291100213517
EPS_1 = 0.2687708899786483
S_2 = 2.5267047923348027
E_2 = 0.8519856985278291
EPS_2 = 0.14801430147217087


In [58]:
#1-Количество входов и выходов могут быть различны
#2-Реализовать расчёт промежуточных слоёв